<a href="https://colab.research.google.com/github/DmitryMok/od_helper/blob/main/prepare_for_od.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модуль для анализа и конвертации датасета изображения

1. Анализ датасета (размеры и ориентация изображений)
2. Анализ аннотаций (наличие аннотаций для всех изображений)
3. Конвертация в формат yolo
4. Фильтрация изображений (по размерам, количеству изображений и т.п.)

## Анализ датасета

In [1]:
import os

In [5]:
!git clone https://github.com/DmitryMok/wine_helper # вспомогательные функции
from wine_helper.prepare_data_helper import *

Cloning into 'wine_helper'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 14), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [2]:
DIR_IMG = '/content/sample_data'    # путь в папке с изображениями
DIR_LAB = ''  # путь к папке с аннотациями


In [3]:
def check_dir(dir_img=DIR_IMG):
  print(os.listdir(dir_img))

In [4]:
# check_dir()

['README.md', 'anscombe.json', 'california_housing_train.csv', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_test.csv']
